-   How do I perform calculations on rasters and extract pixel values for defined locations?

-   Carry out operations with two rasters using Python’s built-in math operators.

## Introduction

We often want to combine values of and perform calculations on rasters to create a new output raster. This episode covers how to perform basic math operations using raster datasets. It also illustrates how to match rasters with different resolutions so that they can be used in the same calculation. As an example, we will calculate [a binary classification mask](https://custom-scripts.sentinel-hub.com/sentinel-2/burned_area_ms/) to identify burned area over a satellite scene.

The classification mask requires the following of [the Sentinel-2 bands](https://gisgeography.com/sentinel-2-bands-combinations/) (and derived indices):
* [Normalized difference vegetation index (NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index), derived from the **near-infrared (NIR)** and **red** bands:
$$ NDVI = \frac{NIR - red}{NIR + red} $$
* [Normalized difference water index (NDWI)](https://en.wikipedia.org/wiki/Normalized_difference_water_index), derived from the **green** and **NIR** bands:
$$ NDWI = \frac{green - NIR}{green + NIR} $$
* A custom index derived from two of the **short-wave infrared (SWIR)** bands (with wavelenght ~1600 nm and ~2200 nm, respectively):
$$ INDEX = \frac{SWIR_{16} - SWIR_{22}}{SWIR_{16} + SWIR_{22}}$$
* The **blue**, **NIR**, and **SWIR** (1600 nm) bands.

In the following, we start by computing the NDVI.

## Load and crop the Data

For this episode, we will use one of the Sentinel-2 scenes that we have already used the previous episodes.

Since we want to calculate the NDVI we are going to focus at “red” and “nir”.

Let’s load the rasters with `open_rasterio` using the argument `masked=True`.

In [ ]:
import rioxarray

red = rioxarray.open_rasterio('../data/input_data/sentinel2/red.tif', masked=True, overview_level=1)
nir = rioxarray.open_rasterio('../data/input_data/sentinel2/nir.tif', masked=True, overview_level=1)

Let’s also restrict our analysis to the island of Rhodes - we can extract the bounding box from the vector file written in an earlier episode (note that we need to match the CRS to the one of the raster files):

In [ ]:
import geopandas
rhodes = geopandas.read_file('../data/output_data/rhodes.gpkg')
rhodes_reprojected = rhodes.to_crs(red.rio.crs)
bbox = rhodes_reprojected.total_bounds

In [ ]:
red_clip = red.rio.clip_box(*bbox)
nir_clip = nir.rio.clip_box(*bbox)

We can now plot the two rasters. Using `robust=True` color values are stretched between the 2nd and 98th percentiles of
the data, which results in clearer distinctions between high and low reflectances:

In [ ]:
red_clip.plot(robust=True)

In [ ]:
nir_clip.plot(robust=True)

The burned area is immediately evident as a dark spot in the NIR wavelength, due to the lack of reflection from the vegetation. 

## Raster Math

We can perform raster calculations by subtracting (or adding,
multiplying, etc.) two rasters. In the geospatial world, we call this
“raster math”, and typically it refers to operations on rasters that
have the same width and height (including `nodata` pixels).
We can check the shapes of the two rasters in the following way:

In [ ]:
print(red_clip.shape, nir_clip.shape)

The shapes of the two rasters match (and, not shown, the coordinates and the CRSs match too). 

Let’s now compute the NDVI as a new raster using the formula presented above.
We’ll use `rioxarray` objects so that we can easily plot our result and keep
track of the metadata.

In [ ]:
ndvi = (nir_clip - red_clip)/ (nir_clip + red_clip)

We can now plot the output NDVI:

In [ ]:
ndvi.plot()

Notice that the range of values for the output NDVI is between -1 and 1.
Does this make sense for the computed value?

Maps are great, but it can also be informative to plot histograms of values to better understand the distribution. We can accomplish this using a built-in xarray method we have already been using: `plot.hist()`

In [ ]:
ndvi.plot.hist()

We see that most of the values are included in the range 0 to 1, which we can use to limit the plotted range:

In [ ]:
ndvi.plot(vmin=0, vmax=1)

### Exercise: NDWI and custom index to detect burned areas

Calculate the other two indices required to compute the burned area classification mask, specifically:

* The [normalized difference water index (NDWI)](https://en.wikipedia.org/wiki/Normalized_difference_water_index), derived from the **green** and **NIR** bands (with band ID "green" and "nir", respectively):
$$ NDWI = \frac{green - NIR}{green + NIR} $$
* A custom index derived from the  1600 nm and the 2200 nm **short-wave infrared (SWIR)** bands ( "swir16" and "swir22", respectively):
$$ INDEX = \frac{SWIR_{16} - SWIR_{22}}{SWIR_{16} + SWIR_{22}}$$

What challenge do you foresee in combining the information from all computed indices?



https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/burned_area_ms/

In [ ]:
def get_band_and_clip(path, band, bbox, ov_level=1):
    band = rioxarray.open_rasterio(path+band, masked=True, overview_level=ov_level)
    return band.rio.clip_box(*bbox)

In [ ]:
data_path = '../data/input_data/sentinel2/' 
green_clip = get_band_and_clip(data_path, 'green.tif', bbox)
swir16_clip = get_band_and_clip(data_path, 'swir16.tif', bbox, ov_level=0) # Use a higher ov level since this is already low resolution
swir22_clip = get_band_and_clip(data_path, 'swir22.tif', bbox, ov_level=0) # Use a higher ov level since this is already low resolution

In [ ]:
ndwi = (green_clip - nir_clip)/(green_clip + nir_clip)
index = (swir16_clip - swir22_clip)/(swir16_clip + swir22_clip)

In [ ]:
ndwi.plot(robust=True)

In [ ]:
index.plot(robust=True)

The challenge in combining the different indices is that the SWIR bands (and thus the derived custom index) have lower resolution:

In [ ]:
ndvi.rio.resolution(), ndwi.rio.resolution(), index.rio.resolution()

We solve the issue by using the `reproject_match` method, which reprojects, clips and match the resolution of a raster using another raster as a template. We use the NDVI raster as a template, and match `index` and `swir16` to its resolution and extent: 

In [ ]:
index_match = index.rio.reproject_match(ndvi)
swir16_clip_match = swir16_clip.rio.reproject_match(ndvi)

In [ ]:
index_match.rio.resolution(), swir16_clip_match.rio.resolution()

Finally, we also fetch the blue band data:

In [ ]:
blue_clip = get_band_and_clip(data_path, 'blue.tif', bbox)

We can now go ahead and compute the binary classification mask for burned areas. Note that we need to convert the unit of the Sentinel-2 bands [from digital numbers to reflectance](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/#units) (this is achieved by dividing by 10,000):

In [ ]:
burned = (
    (ndvi <= 0.3) & 
    (ndwi <= 0.1) &
    ((index_match + nir_clip/10_000) <= 0.1) &
    ((blue_clip/10_000) <= 0.1) & 
    ((swir16_clip_match/10_000) >= 0.1)
)

The classification mask has a single element along the "band" axis, we can drop this dimension in the following way (it makes it easier to work with data with comparable shape (it removes the dummy band in the array): 

In [ ]:
# A method you can perform to get rid of the dummy band it creates; remove dimensions of size 1 from a DataArray or Dataset. The raster image `burned` is a 3D image with a "band" dimension.
#To create the grid space, we only need the two spatial dimensions. We can used `.squeeze()` to drop the band dimension:
burned = burned.squeeze() 

Let's now fetch and visualize the true color image of Rhodes, after coloring the pixels identified as burned area in red:  

In [ ]:
visual = rioxarray.open_rasterio(data_path+"visual.tif", overview_level=1)
visual_clip = visual.rio.clip_box(*bbox)

In [ ]:
# set red channel to max (255), green and blue channels to min (0).
visual_clip[0] = visual_clip[0].where(~burned, 255) #the ~ operator in Python, especially in a NumPy or xarray context, is used for bitwise negation, and it’s often used to invert boolean arrays. ~burned: This is where the magic happens — it inverts the boolean mask.
visual_clip[1:3] = visual_clip[1:3].where(~burned, 0)

In [ ]:
visual_clip.plot.imshow()

We can save the burned classification mask to disk after converting booleans to integers:

In [ ]:
burned.rio.to_raster('../data/output_data/burned.tif', dtype='int8')